# kTextAugmentation

이 노트북은 kTextAugmentation 패키지를 사용하기 위한 예시입니다.
documentation을 완성하기전까지는 이 노트북을 위주로 설명할 것 같습니다

### TextAugmentation() 사용하기
패키지 0.1.9 버젼부턴 기본적으로 TextAugmentation() 을 사용하여 처리하는 것을 권장합니다. multiprocessing 을 이용하여 대용량의 데이터를 빠르게 처리할 수 있도록 만들었습니다. 


In [1]:
from ktextaug import TextAugmentation

sample_text = '달리는 기차 위에 중립은 없다. 미국의 사회 운동가이자 역사학자인 하워드 진이 남긴 격언이다.'
sample_texts = ['프로그램 개발이 끝나고 서비스가 진행된다.', '도움말을 보고 싶다면 --help를 입력하면 된다.']
agent = TextAugmentation(tokenize_fn="mecab",
                        num_processes=1) # num_process 가 -1 일시 자동으로 가능한 process의 절반으로 할당
print(agent.generate(sample_text))     # default is back_translation
print(agent.generate(sample_texts))

type check False
달리는 기차에는 중립이 없습니다. 이것은 미국 사회 운동가이자 역사가 하워드 진의 속담입니다.



['프로그램 개발이 완료되면 서비스가 진행됩니다.', '도움말을 보려면 --help를 입력하십시오.']


### 직접 import 해서 사용하기
0.1.8 에서 사용하던 대로 직접 원하는 기능을 import 할 수 있습니다.

현재 불러올 수 있는 함수는 다음과 같습니다. 
"back_translate",
"noise_add",
"random_insert",
"random_delete",
"random_swap",
"synonym_replace",

In [9]:
from ktextaug import random_swap
from random import Random

text = "이 문장은 변형적 데이터 증강기법의 예시 문장입니다."
tokenizer = bring_it_your_own   # 토크나이저는 어떤 토크나이저를 사용하더라도 상관없습니다.

result = random_swap(text_or_words=text,
                     tokenize_fn=tokenizer.tokenize, 
                     rng=Random(seed=2021),
                     n_swap=2) # random_swap 고유 파라메터. 토큰 시퀀스 내 두 단어의 위치를 변경하는 작업(random swap)을 2회 시행합니다. 
print(result)
# ['이', '문장', '은', '예시', '적', '데이터', '기법', '증강', '의', '문장', '변형', '입니다', '.']

NameError: name 'bring_it_your_own' is not defined

### tokenize_fn 

ktextaug패키지는 tokenizer 내부에서 따로 wrapping 하지 않으며, <class 'function'> 형태의 tokenize 함수를 받습니다. 이미 너무 잘 만들어진 tokenizer 패키지들이 많은데, 여기서 불필요하게 wrapping 할 필요가 없다고 판단했습니다. 단, 사용자들이 원할 경우 사용할 수 있도록 get_tokenize_fn 를 구현해두었습니다. mecan, komoran이 설치되어있는 경우 사용할 수 있으며, subword의 경우 transformers 의 BertTokenizer을 가져와 사용합니다.

(subword 경우 노이즈와 관련된 vocab을 구축해놓았기 때문에 다른 토크나이저에 비해 noise_add에서 훨씬 강건합니다.)

In [13]:
from ktextaug.tokenization_utils import get_tokenize_fn

# get_tokenize_fn 함수의 사용예시
tokenize_fn = get_tokenize_fn("mecab")
print(type(tokenize_fn))
print(tokenize_fn("나는 학교에서 밥을 먹었다."))
tokenize_fn = get_tokenize_fn("komoran")
print(tokenize_fn("나는 학교에서 밥을 먹었다."))
tokenize_fn = get_tokenize_fn("subword")
print(tokenize_fn("나는 학교에서 밥을 먹었다."))

# 이러한 <class 'function'> 또는 <class 'method'> 형태의 함수들은 패키지 내 함수나 TextAugmentation 클래스에 사용됩니다.

agent = TextAugmentation(tokenize_fn=tokenize_fn,
#                         tokenize_fn="mecab",
                        num_processes=1) 

result = random_swap(text_or_words=text,
                     tokenize_fn=lambda x: x.split(" "), # lambda로도 사용 가능
                     rng=Random(seed=2021),
                     n_swap=2)

<class 'method'>
['나', '는', '학교', '에서', '밥', '을', '먹', '었', '다', '.']


FileNotFoundError: [Errno 2] No such file or directory: 'java'

### noise_generation 모듈 사용법

노이즈 생성은 @hkjeon13(전현규) 의 노이즈 생성을 따랐습니다

https://github.com/hkjeon13/noising-korean

노이즈를 생성하는 방법은 총 3가지가 구현되어 있습니다.

"jamo_split": 자모 분리(alphabet separation)에 의한 노이즈 추가 방법. 글자의 자음과 모음을 분리합니다. 단, 가독성을 위해 종성이 없으며 중성이  'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅗ' 가 아닐 경우 실행합니다(예: 안녕하세요 > 안녕ㅎㅏㅅㅔ요)

"vowel_change": 모음 변형에 의한 노이즈 추가 방법. 글자의 모음을 변형시킵니다. 단, 가독성을 위해 종성이 없으며 중성이 'ㅏ', 'ㅑ', 'ㅓ', 'ㅕ', 'ㅗ', 'ㅛ', 'ㅜ', 'ㅠ' 일 경우 실행합니다(예: 안녕하세요 > 안녕햐세오).

"phonological_change": 음운변화에 의한 노이즈 추가 방법. 발음을 바탕으로 단어를 변형시킵니다(너무 닮았다 > 너무 달맜다).

In [3]:
import noise_generation

text = '행복한 가정은 모두가 닮았지만, 불행한 가정은 모두 저마다의 이유로 불행하다.'
agent = TextAugmentation(tokenize_fn="mecab",
                        num_process=1) 
print(agent.generate(text,
                     mode="noise_add",
                     noise_mode="jamo_split"))     
# >> 행복한 ㄱㅏ정은 모두ㄱㅏ 닮았ㅈㅣ만, 불행한 ㄱㅏ정은 모두 ㅈㅓㅁㅏㄷㅏ의 ㅇㅣ유로 불행ㅎㅏㄷㅏ.

ModuleNotFoundError: No module named 'noise_generation'



**노이즈 생성의 변형 예시**

```python
text = '행복한 가정은 모두가 닮았지만, 불행한 가정은 모두 저마다의 이유로 불행하다.'
[original]  행복한 가정은 모두가 닮았지만, 불행한 가정은 모두 저마다의 이유로 불행하다.

[jamo_split, prob=1] 행복한 ㄱㅏ정은 모두ㄱㅏ 닮았ㅈㅣ만, 불행한 ㄱㅏ정은 모두 ㅈㅓㅁㅏㄷㅏ의 ㅇㅣ유로 불행ㅎㅏㄷㅏ.

[vowel_change, prob=1] 행복한 갸정은 묘듀갸 닮았지만, 불행한 갸정은 묘듀 져먀댜의 이우료 불행햐댜.

[phonological_change, prob=1] 행복한 가정은 모두가 달맜지만, 불행한 가정은 모두 저마다의 이유로 불행하다.
```

#### 기타

- 'phonological_change' 방법은 현재 비음화, 유음화, 구개음화, 연음 등을 구현하고 있으며, 추후 확대될 예정입니다(누락된 규칙이 있을 수 있으니, 발견 시 피드백 주시면 감사하겠습니다).
- prob는 변형 가능한 글자들에 대해서 해당 확률만큼 확률적으로 실행됩니다(prob가 1이라고 해서 모든 텍스트가 변경되는 것이 아닙니다).